In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [2]:
# df = pd.read_csv("dataset/hindi_english_parallel.csv")
# df

In [45]:
import torch
from datasets import load_dataset

from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace
from dataloader.dataloader import NMTDataset
%reload_ext autoreload
%autoreload 2

In [51]:
train_dataset = load_dataset('cfilt/iitb-english-hindi', split="train[:36000]")
train_dataset

test_dataset = load_dataset('cfilt/iitb-english-hindi', split="test")
len(test_dataset)

Repo card metadata block was not found. Setting CardData to empty.
Repo card metadata block was not found. Setting CardData to empty.


2507

In [52]:
len(train_dataset)

36000

In [6]:
val_dataset = load_dataset('cfilt/iitb-english-hindi', split="validation")
len(val_dataset)

Repo card metadata block was not found. Setting CardData to empty.


520

In [53]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=len(train_dataset))
# val_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=len(val_dataset))
# test_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=len(test_dataset))

In [12]:
B = 1000
tr = next(iter(train_dataloader))
vl = next(iter(val_dataloader))
tt = next(iter(test_dataloader))

In [13]:
# Apply tokenizer & make vocab
tokenizer = Tokenizer(WordLevel(unk_token="<unk>"))
tokenizer.pre_tokenizer = Whitespace()
trainer = WordLevelTrainer(special_tokens=['<unk>', '<pad>', '<sos>', '<eos>'], min_frequency=2)
tokenizer.train_from_iterator(tr['translation']['hi'] + vl['translation']['hi']+ tt['translation']['hi'], trainer=trainer)

# save tokenizer
tokenizer.save("dataset/vocab_hi.json")

# load tokenizer
# tokenizer = Tokenizer.from_file(str(tokenizer_path))

In [14]:
# Apply tokenizer & make vocab
lang = "en"
tokenizer = Tokenizer(WordLevel(unk_token="<unk>"))
tokenizer.pre_tokenizer = Whitespace()
trainer = WordLevelTrainer(special_tokens=['<unk>', '<pad>', '<sos>', '<eos>'], min_frequency=2)
tokenizer.train_from_iterator(tr['translation'][lang] + vl['translation'][lang]+ tt['translation'][lang], trainer=trainer)

# save tokenizer
tokenizer.save(f"dataset/vocab_{lang}.json")

# load tokenizer
# tokenizer = Tokenizer.from_file(str(tokenizer_path))

In [15]:
tokenizer_path = f"dataset/vocab_hi.json"
tokenizer_hi = Tokenizer.from_file(tokenizer_path)

tokenizer_path = f"dataset/vocab_en.json"
tokenizer_en = Tokenizer.from_file(tokenizer_path)

In [43]:
dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=len(train_dataset))
tokenizer_tar = tokenizer_hi
tokenizer_src = tokenizer_en 
data = []

max_src, max_tar = 0, 0
x = next(iter(dataloader))["translation"]
for src_sent, tar_sent in zip(x["en"], x["hi"]):
    src_tokens = [tokenizer_src.token_to_id("<sos>")] + tokenizer_src.encode(src_sent).ids + [tokenizer_src.token_to_id("<eos>")]
    tar_tokens = [tokenizer_tar.token_to_id("<sos>")] + tokenizer_tar.encode(tar_sent).ids + [tokenizer_tar.token_to_id("<eos>")]
    data.append((src_tokens, tar_tokens))
    
    max_src, max_tar = max(max_src, len(src_tokens)), max(max_tar, len(tar_tokens))

In [57]:
tokenizer_en.get_vocab_size()

3388

In [54]:
ds = NMTDataset(tokenizer_en, tokenizer_hi, train_dataloader)

In [55]:
len(ds)

82

In [44]:
max_src, max_tar

(82, 100)

In [54]:
x = next(iter(dataloader))

In [55]:
x['translation']['hi'][:4]

["महानगर पालिका अंतर्गत दत्तात्रय नगर माध्यमिक स्कूल के विद्यार्थियों ने काल्पनिक किला 'दत्तगढ़' बनाकर अपनी कल्पनाशक्ति का परिचय दिया।",
 'प्रधानाध्यापक संध्या मेडपल्लीवार के प्रोत्साहित करने पर शिक्षकों व विद्यार्थियों ने मिट्टïी से किले का निर्माण किया।',
 'मनपा शिक्षक संघ के अध्यक्ष राजेश गवरे ने स्कूल को भेंट देकर सराहना की।',
 'किले का परीक्षण रमेश सातपुते ने किया।']

In [56]:
x['translation']['en'][:4]

['Students of the Dattatreya city Municipal corporation secondary school demonstrated their imagination power by creating the fictitious fort "Duttgarh".',
 'With encouragement from Principal Sandhya Medpallivaar the teachers and students built the fort out of clay.',
 'Rajesh Gavre, the President of the MNPA teachers association, honoured the school by presenting the award.',
 'Ramesh Saatpute examined the fort.']

In [57]:
import torch
from torchtext.data.utils import get_tokenizer
from torch.utils.data import Dataset, DataLoader
from collections import Counter
from torchtext.vocab import vocab
from sklearn.model_selection import train_test_split
import pandas as pd
from torch.nn.utils.rnn import pad_sequence
gpu_device = torch.device("cuda")



In [58]:
# !pip install spacy
# !python3 -m spacy download en_core_web_sm
# !pip install inltk
# import nltk
# nltk.download('punkt')

### Building english vocab using Counter()

In [59]:
en_tokenizer = get_tokenizer('spacy', language='en')   
counter = Counter() # dict of {token: Freq}     
for source in x['translation']['en']:
    counter.update(en_tokenizer(source))

In [60]:
voc_en = vocab(counter, specials=['<unk>', '<pad>', '<sos>', '<eos>'])

In [61]:
#voc_en["karm"] # D:not able to handle OoV
voc_en["is"]

162

In [66]:
from tokenizers import Tokenizer
tokenizer = Tokenizer.from_file("dataset/tokenizer_hi.json")

### Building hindi vocab using inltk and Counter()

In [62]:
x['translation']['hi'][0]

"महानगर पालिका अंतर्गत दत्तात्रय नगर माध्यमिक स्कूल के विद्यार्थियों ने काल्पनिक किला 'दत्तगढ़' बनाकर अपनी कल्पनाशक्ति का परिचय दिया।"

In [71]:
tokenizer.token_to_id("प्रोजेक्ट")

70

In [63]:
tokenize(x['translation']['hi'][0], "hi")

['▁महानगर',
 '▁पालिका',
 '▁अंतर्गत',
 '▁दत्ता',
 'त्रय',
 '▁नगर',
 '▁माध्यमिक',
 '▁स्कूल',
 '▁के',
 '▁विद्यार्थियों',
 '▁ने',
 '▁काल्पनिक',
 '▁किला',
 "▁'",
 'दत्त',
 'गढ़',
 "'",
 '▁बनाकर',
 '▁अपनी',
 '▁कल्पना',
 'शक्ति',
 '▁का',
 '▁परिचय',
 '▁दिया',
 '।']

In [64]:
counter = Counter() # dict of {token: Freq}     
for source in x['translation']['hi']:
    counter.update(tokenize(source, "hi"))
voc_hi = vocab(counter, specials=['<unk>', '<pad>', '<sos>', '<eos>']) 

In [41]:
voc_hi['▁दिया']

27

### Token ids

In [45]:
token_ids = []
for source, target in zip(x['translation']['en'], x['translation']['hi']):
    source_tensor = torch.tensor([voc_en["<sos>"]] + [voc_en[word] for word in en_tokenizer(source)] + [voc_en["<eos>"]])
    target_tensor = torch.tensor([voc_hi["<sos>"]] + [voc_hi[word] for word in tokenize(target, "hi")] + [voc_hi["<eos>"]])    
    token_ids.append((source_tensor, target_tensor))

In [47]:
token_ids[:2]

[(tensor([ 2,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,  6, 19,
          20, 21, 22, 21, 23,  3]),
  tensor([ 2,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
          21, 22, 23, 24, 25, 26, 27, 28,  3])),
 (tensor([ 2, 24, 25, 26, 27, 28, 29,  6, 30, 31, 32, 33,  6, 20, 34,  5, 35, 23,
           3]),
  tensor([ 2, 29, 30, 31, 32, 33, 34, 12, 35, 36, 37, 38, 39, 13, 14, 40, 41, 42,
          43, 44, 45, 25, 46, 47, 28,  3]))]

In [ ]:
import torch
from torchtext.data.utils import get_tokenizer
from torch.utils.data import Dataset, DataLoader
from collections import Counter
from torchtext.vocab import vocab
from sklearn.model_selection import train_test_split
import pandas as pd
from torch.nn.utils.rnn import pad_sequence
gpu_device = torch.device("cuda")

# todos 1) Make generalize tokenizer 2) make preprocessing different so we can use while custom dataset
class DateDataset(Dataset):
    def __init__(self, DATASET_PATH = "dataset/Assignment2aDataset.txt", split="train"):
        df = pd.read_csv(DATASET_PATH, names = ["source", "target"])
        df["source"] = df["source"].apply(lambda x: x.strip()[1:-1].replace("/", "-"))
        df["target"] = df["target"].apply(lambda x: x.strip()[1:-1])
        df_train, df_test = train_test_split(df, random_state=42, test_size=0.1)
        
        # tokenize
        en_tokenizer = get_tokenizer('spacy', language='en')   
        counter = Counter() # dict of {token: Freq}     
        for source in df["source"]:
            counter.update(en_tokenizer(source))

        for source in df["target"]:
            counter.update(en_tokenizer(source))
        
        voc = vocab(counter, specials=['<unk>', '<pad>', '<bos>', '<eos>'])    
        
        # create data
        if split == "train":
            self.data_df = df_train
        else:
            self.data_df = df_test
            
        data = []
        for (source, target) in zip(self.data_df["source"], self.data_df["target"]):
            s_tensor_ = torch.tensor([voc[token] for token in en_tokenizer(source)])
            t_tensor_ = torch.tensor([voc[token] for token in en_tokenizer(target)])
            data.append((s_tensor_, t_tensor_))
        
        self.voc = voc
        self.data = data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]\
        

def generate_batch(data_batch, special_tokens):
    BOS_IDX = special_tokens["BOS_IDX"]
    PAD_IDX = special_tokens["PAD_IDX"]
    EOS_IDX = special_tokens["EOS_IDX"]

    s_batch, t_batch = [], []
    for (s_item, t_item) in data_batch:
        s_batch.append(torch.cat([torch.tensor([BOS_IDX]), s_item, torch.tensor([EOS_IDX])], dim=0))
        t_batch.append(torch.cat([torch.tensor([BOS_IDX]), t_item, torch.tensor([EOS_IDX])], dim=0))
        
    s_batch = pad_sequence(s_batch, padding_value=PAD_IDX)
    return s_batch.T.to(gpu_device), torch.stack(t_batch).to(gpu_device)

def get_dataloader(split="train", batch_size=4000):
    dataset = DateDataset(split=split)
    special_tokens = {}
    special_tokens["BOS_IDX"] = dataset.voc["<bos>"]
    special_tokens["EOS_IDX"] = dataset.voc["<eos>"]
    special_tokens["PAD_IDX"] = dataset.voc["<pad>"]

    dataloader = DataLoader(dataset, batch_size=batch_size, collate_fn=lambda batch, 
                            sp_tokens = special_tokens : generate_batch(batch, sp_tokens))

    return dataset, dataloader


In [ ]:
# tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
# dataset = dataset.map(lambda e: tokenizer(e['sentence1']), batched=True)
# dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
# dataloader = torch.utils.data.DataLoader(dataset, batch_size=32)
# next(iter(dataloader))